In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

In [11]:
import ray

import pandas as pd
import numpy as np
import yaml
import json
import os
from flatten_dict import unflatten, flatten

from SepMe.utils.filter_utils import *
from SepMe.utils.preprocessing_utils import *

from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.preprocessing import minmax_scale, LabelEncoder
#import umap
import psutil


print("Number of processors: ", psutil.cpu_count())
os.getcwd()

/home/morarica/anaconda3/envs/SepMe/lib/python3.7/site-packages/numba/types/containers.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


Number of processors:  20


'/home/morarica/Developer/SepMe/Jupyters'

In [4]:
ray.init(num_cpus=psutil.cpu_count())

2019-11-22 17:23:04,196	INFO resource_spec.py:205 -- Starting Ray with 13.09 GiB memory available for workers and up to 6.56 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '129.69.205.196',
 'redis_address': '129.69.205.196:54804',
 'object_store_address': '/tmp/ray/session_2019-11-22_17-23-04_195082_8600/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-22_17-23-04_195082_8600/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-11-22_17-23-04_195082_8600'}

## 1. Getting the human annotations

In [6]:
folder = '../data/SPLOM_data/SepMe/Data/Human_judgment/'
print(len(sorted(os.listdir(folder))))
df_a = []
df_m = []
for i, file in enumerate(sorted(os.listdir(folder))):
    if file.endswith('csv'):
        #print(file)
        df = pd.read_csv(folder+file, header=None)
        df.columns = np.array(df.columns)+1
        
        if len(df)>4:
            #print(df)
            df = df.head(4)
        
        df.index = ['PCA', 'RobPCA', 'GlimmerMDS', 'tSNE']
        
        person = file.split('.csv')[0].split('-')[-1]
        dd ={}
        dd[file.split('-')[0]] = df.to_dict('index')
        df1 = pd.DataFrame.from_dict(flatten(dd, underscore_reducer), orient='index', columns = [person])
        
        #print(person + '  ' + file)
        if person =='M':
            df_m.append(df1)
        else:
            df_a.append(df1)

dfmm = pd.concat(df_m, axis=0)
dfaa = pd.concat(df_a, axis=0)

df = pd.concat([dfmm, dfaa], axis=1)


df['class'] = [ii.split('_')[-1] for ii in df.index]
df.dropna(axis=0, inplace=True)

df = df.astype({'A': 'int32', 'M': 'int32'})
print(df.shape)

153
(1669, 3)


In [7]:
df['dim_x'] = 1
df['dim_y'] = 2
df = df[df.A.between(1,5) & df.M.between(1,5)]
df = df.reset_index()
print(df.shape)
df.head()


names = ['_'.join(row.split('_')[:-1]) for row in list(df['index'])]
#names = list(set(names))

df['fileName'] = names

(1648, 6)


In [8]:
df.to_csv('/home/morarica/Developer/SepMe/data/orig_data/human_reduced_results.csv', index=False)

## 2. Plot inputs

In [14]:
def get_dimred_data(df, input_folder, save_folder, fig_folder):
    found = 0
    not_found = []
    names = ['_'.join(row.split('_')[:-1]) for row in list(df['index'])]
    names = list(set(names))
    
    for i, nn in enumerate(names):
        process_one_dimred.remote(i, names, nn, input_folder, df, save_folder, fig_folder)
    return 

In [15]:
input_folder = '/home/morarica/Developer/SepMe/data/orig_data/input_data/Reduced_orig_data/'
save_folder = '/home/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/'
fig_folder = '/home/morarica/Developer/SepMe/data/orig_data/figures/reduced_data/'

get_dimred_data(df, input_folder, save_folder, fig_folder)

(pid=8682) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8670) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8677) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8669) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8681) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8666) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8667) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8672) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8671) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8680) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8664) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8665) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8668) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8674) SepMe - INFO - MLFLOW_TRACKING_URI - http://localhost:5000
(pid=8675) SepMe - I

(pid=8680) Processing file 15/216. Name: cars03Cropped_d1_RobPCA
(pid=8677) Processing file 34/216. Name: grid6_4d_tSNE
(pid=8677) File not found: grid6_4d_tSNE_2.csv
(pid=8677) Processing file 35/216. Name: breast_tSNE
(pid=8677) File not found: breast_tSNE_2.csv
(pid=8677) Processing file 36/216. Name: ms_interleaved_60_120_3d_50_centeredClusters_tSNE
(pid=8677) File not found: ms_interleaved_60_120_3d_50_centeredClusters_tSNE_2.csv
(pid=8677) Processing file 37/216. Name: swanson_GlimmerMDS
(pid=8669) Processing file 38/216. Name: n500_tSNE
(pid=8669) File not found: n500_tSNE_2.csv
(pid=8669) Processing file 39/216. Name: n500_GlimmerMDS
(pid=8669) File not found: n500_GlimmerMDS_2.csv
(pid=8669) Processing file 40/216. Name: wdbc_class_1_2_RobPCA
(pid=8675) Processing file 41/216. Name: ms_interleaved_40_80_3d_0_PCA
(pid=8682) Processing file 42/216. Name: spambase_PCA
(pid=8667) Processing file 43/216. Name: tse300_tSNE
(pid=8667) File not found: tse300_tSNE_2.csv
(pid=8667) Proc

(pid=8678) Processing file 110/216. Name: fisheries_clusteredByEscapementTarget_PCA
(pid=8678) Classes: 11
(pid=8677) Processing file 111/216. Name: JavierGeneratedData_3dinterleaved_3classes_PCA
(pid=8664) Processing file 112/216. Name: interleaved_100_200_6d_0_notcramped_notrotated_PCA
(pid=8675) Processing file 113/216. Name: n100_GlimmerMDS
(pid=8675) File not found: n100_GlimmerMDS_2.csv
(pid=8675) Processing file 114/216. Name: ms_interleaved_120_240_3d_25_centeredClusters_GlimmerMDS
(pid=8678) Processing file 115/216. Name: ms_interleaved_60_120_3d_0_centeredClusters_GlimmerMDS
(pid=8673) Processing file 116/216. Name: interleaved_100_500_3_25_cramped_rotated_tSNE
(pid=8673) File not found: interleaved_100_500_3_25_cramped_rotated_tSNE_2.csv
(pid=8673) Processing file 117/216. Name: ms_interleaved_40_80_3d_50_RobPCA
(pid=8676) Processing file 118/216. Name: JavierGeneratedData_3dinterleaved_5classes_GlimmerMDS
(pid=8679) Processing file 119/216. Name: interleaved_100_200_5d_0_no

(pid=8670) Processing file 186/216. Name: shuttle_big_PCA
(pid=8676) Processing file 187/216. Name: JavierGeneratedData_3dinterleaved_3classes_GlimmerMDS
(pid=8681) Processing file 188/216. Name: efashion_RobPCA
(pid=8671) Processing file 189/216. Name: italianwines_PCA
(pid=8671) Classes: 13
(pid=8671) Processing file 190/216. Name: interleaved_100_200_4d_0_notcramped_notrotated_tSNE
(pid=8671) File not found: interleaved_100_200_4d_0_notcramped_notrotated_tSNE_2.csv
(pid=8671) Processing file 191/216. Name: ms_interleaved_60_120_3d_0_centeredClusters_PCA
(pid=8667) Processing file 192/216. Name: ms_interleaved_60_120_3d_50_centeredClusters_GlimmerMDS
(pid=8664) Processing file 193/216. Name: n100_RobPCA
(pid=8664) File not found: n100_RobPCA_data.csv
(pid=8664) Processing file 194/216. Name: n100_tSNE
(pid=8664) File not found: n100_tSNE_2.csv
(pid=8664) Processing file 195/216. Name: interleaved_250_500_3_0_notCramped_rotated_tSNE
(pid=8664) File not found: interleaved_250_500_3_0_n

## 3. Plot dimensions in original data

In [13]:
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE, LocallyLinearEmbedding, MDS

@ray.remote
def add_dimreds(orig_dir, save_dir, file):
    
    if os.path.exists(save_dir + file):
        #print('File was already processed. Skipping: {}'.format(save_dir + file))
        return pd.DataFrame({})
    
    df = pd.read_csv(orig_dir + file)
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    ndf = df.select_dtypes(include=numerics)
    ndf = ndf.iloc[:,:-1].copy()
    
    
    
    print('Applying UMAP..')
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(ndf)
    df['umap1'] = embedding[:,0]
    df['umap2'] = embedding[:,1]
    
    print('Applying PCA..')
    pca = PCA(n_components=3, whiten=True)
    embedding = pca.fit_transform(ndf) 
    df['pca1'] = embedding[:,0]
    df['pca2'] = embedding[:,1]
    
    print('Applying Isomap..')
    iso = Isomap(n_components=2, n_neighbors=3)
    embedding = iso.fit_transform(ndf) 
    df['iso1'] = embedding[:,0]
    df['iso2'] = embedding[:,1]
    
    print('Applying LLE..')
    lle = LocallyLinearEmbedding(n_components=2, eigen_solver='dense')
    embedding = lle.fit_transform(ndf) 
    df['lle1'] = embedding[:,0]
    df['lle2'] = embedding[:,1]
    
    print('Applying tSNE..')
    tsne = TSNE(n_components=2)
    embedding = tsne.fit_transform(ndf) 
    df['tsn1'] = embedding[:,0]
    df['tsn2'] = embedding[:,1]
    
    print('Applying non-metric MDS..')
    mds = MDS(n_components=2, metric=False)
    embedding = mds.fit_transform(ndf) 
    df['mds1'] = embedding[:,0]
    df['mds2'] = embedding[:,1]
    
    dr_cols = ['pca1', 'pca2', 'iso1', 'iso2', 'tsn1', 'tsn2', 'lle1', 'lle2', 'umap1', 'umap2', 'mds1', 'mds2']
    
    df = df[dr_cols + list(ndf.columns) + ['class']]
    df.loc[:, dr_cols+ ndf.columns] = minmax_scale(df.loc[:, dr_cols + ndf.columns])
    
    
    print(df.columns)
    df.to_csv(save_dir+file, index=False)
    
    return df
    
    
@ray.remote
def save_and_plot_all_dimensions(file, orig_dir, save_dir, fig_dir, class_cols):     
    df = pd.read_csv(orig_dir + file)
    
    c1s = []
    for i, c1 in enumerate(df.columns):
        if not np.issubdtype(df[c1].dtype, np.number) or (c1 in class_cols) or i>25:
            continue
            

        c1s.append(c1)
        for i, c2 in enumerate(df.columns[:-1]):
            if (not np.issubdtype(df[c1].dtype, np.number)) or (c2 in c1s) or (c2 in class_cols) or i>25:
                continue

            file_name = save_dir + file.split('.csv')[0] + '_{}-{}'.format(c1,c2) + '.csv'
            fig_name = fig_dir + file.split('.csv')[0] + '_{}-{}'.format(c1,c2) + '.png'

            if os.path.exists(file_name) is True:
                #print('File was already processed. Skipping: {}'.format(file_name.split('/')[-1]))
                continue

            try: 
                print('{} (len: {}): ({}-{})'.format(file, len(df),c1,c2))
                ddf, circles = preprocess_df(df, [c1, c2], size = 0.5, 
                                         occlusion = 0.1, save=file_name, sort=True)
                fig, ax = plt.subplots(figsize = (20,20))
                
                plot_colored_circles(ax, ddf, circles)
                ax.tick_params(axis='both', which='major', labelsize=30)
                
                fig.savefig(fig_name)
                plt.close('all')

            except Exception as e: 
                print('File {} was not processed for columns ({},{}).'.format(file,c1,c2))
                print(e)
                print('')


## 4. Calculate DimRed of Orig Data

In [14]:
# import warnings
# warnings.filterwarnings('ignore')

# orig_dir = "/home/morarica/Developer/SepMe/data/orig_data/input_data/Orig_data/"
# save_dir = "/home/morarica/Developer/SepMe/data/orig_data/input_data/enhanced_data/"
# fig_dir = "/home/morarica/Developer/SepMe/data/orig_data/figures/orig_dim_data/"
# class_cols = ['class']


# files = []
# results = []

# for i, file in enumerate(os.listdir(orig_dir)):
#     if file.endswith('.csv'):
#         files.append(file)
#         print('Processing file {}/{}. Name: {}'.format(i, len(os.listdir(orig_dir)), file))
        
#         results.append(add_dimreds.remote(orig_dir, save_dir, file))

# #results = ray.get(results)




## 5. Compute all scatterplots for enhanced data

In [ ]:
orig_dir = "/home/morarica/Developer/SepMe/data/orig_data/input_data/enhanced_data/"
save_dir = "/home/morarica/Developer/SepMe/data/orig_data/input_data/enhanced_dim_data/"
fig_dir = "/home/morarica/Developer/SepMe/data/orig_data/figures/enhanced_dim_data/"
class_cols = ['class']
results = []

for i, file in enumerate(os.listdir(orig_dir)):
    if file.endswith('.csv'):
        #print('Processing file {}/{}. Name: {}'.format(i, len(os.listdir(orig_dir)), file))
        if file in ['shuttle_small_origClassLabels.csv', 'shuttle_big.csv', 'grid6_5d.csv', 
                    'cars03Cropped_d2.csv', 'swanson.csv', 'spambase.csv', 'cars03Cropped_d3.csv',
                    'ionosphere.csv']:
            continue
        
        results.append(save_and_plot_all_dimensions.remote(file, orig_dir, save_dir, fig_dir, class_cols))
        
results = ray.get(results)

(pid=26454) ionosphere.csv (len: 351): (pca1-D1)
(pid=26443) bbdm13.csv (len: 200): (pca1-V4)
(pid=26443) File bbdm13.csv was not processed for columns (pca1,V4).
(pid=26443) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26443) 
(pid=26443) bbdm13.csv (len: 200): (pca1-V7)
(pid=26443) File bbdm13.csv was not processed for columns (pca1,V7).
(pid=26443) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26443) 
(pid=26438) ecoliproteins.csv (len: 332): (pca1-d3)
(pid=26438) File ecoliproteins.csv was not processed for columns (pca1,d3).
(pid=26438) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26438) 
(pid=26438) ecoliproteins.csv (len: 332): (pca1-d4)
(pid=26438) File ecoliproteins.csv was not processed for columns (pca1,d4).
(pid=26438) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26438) 
(pid=26454) File ionosphere.csv was not processed for c

(pid=26454) File ionosphere.csv was not processed for columns (tsn2,D2).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (lle1-D1)
(pid=26454) File ionosphere.csv was not processed for columns (lle1,D1).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (lle1-D2)
(pid=26454) File ionosphere.csv was not processed for columns (lle1,D2).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (lle2-D1)
(pid=26454) File ionosphere.csv was not processed for columns (lle2,D1).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (lle2-D2)
(pid=26452) swissroll.csv (len: 1809): (pca1-Col2)
(pid=26452) Classes: 16
(pid=26443) File bbdm13.c

(pid=26454) File ionosphere.csv was not processed for columns (D1,D11).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (D1-D12)
(pid=26454) File ionosphere.csv was not processed for columns (D1,D12).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (D1-D13)
(pid=26454) File ionosphere.csv was not processed for columns (D1,D13).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (D1-D14)
(pid=26454) File ionosphere.csv was not processed for columns (D1,D14).
(pid=26454) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26454) 
(pid=26454) ionosphere.csv (len: 351): (D2-D3)
(pid=26454) File ionosphere.csv was not processed for columns (D2,D3).
(pid=26454) Found array with 0 sample(

(pid=26441) cars03Cropped_d1.csv (len: 7404): (d6-d11)
(pid=26448) File cars03Cropped_d3.csv was not processed for columns (iso1,d1).
(pid=26448) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26448) 
(pid=26448) cars03Cropped_d3.csv (len: 7404): (iso2-d1)
(pid=26448) Classes: 12
(pid=26448) File cars03Cropped_d3.csv was not processed for columns (iso2,d1).
(pid=26448) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26448) 
(pid=26448) cars03Cropped_d3.csv (len: 7404): (tsn1-d1)
(pid=26448) Classes: 12
(pid=26448) File cars03Cropped_d3.csv was not processed for columns (tsn1,d1).
(pid=26448) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26448) 
(pid=26448) cars03Cropped_d3.csv (len: 7404): (tsn2-d1)
(pid=26448) Classes: 12
(pid=26448) File cars03Cropped_d3.csv was not processed for columns (tsn2,d1).
(pid=26448) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is 

(pid=26442) File spambase.csv was not processed for columns (mds2,d4).
(pid=26442) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26442) 
(pid=26442) spambase.csv (len: 4601): (d1-d4)
(pid=26442) File spambase.csv was not processed for columns (d1,d4).
(pid=26442) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26442) 
(pid=26442) spambase.csv (len: 4601): (d2-d4)
(pid=26442) File spambase.csv was not processed for columns (d2,d4).
(pid=26442) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26442) 
(pid=26442) spambase.csv (len: 4601): (d3-d4)
(pid=26442) File spambase.csv was not processed for columns (d3,d4).
(pid=26442) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26442) 
(pid=26442) spambase.csv (len: 4601): (d4-d5)
(pid=26442) File spambase.csv was not processed for columns (d4,d5).
(pid=26442) Found array with 0 sample(s) (shape=(0, 2)) w

(pid=26444) File cars03Cropped_d2.csv was not processed for columns (d1,d19).
(pid=26444) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26444) 
(pid=26444) cars03Cropped_d2.csv (len: 7404): (d1-d11)
(pid=26444) Classes: 53
(pid=26444) File cars03Cropped_d2.csv was not processed for columns (d1,d11).
(pid=26444) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26444) 
(pid=26444) cars03Cropped_d2.csv (len: 7404): (d1-d12)
(pid=26444) Classes: 53
(pid=26444) File cars03Cropped_d2.csv was not processed for columns (d1,d12).
(pid=26444) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26444) 
(pid=26444) cars03Cropped_d2.csv (len: 7404): (d1-d13)
(pid=26444) Classes: 53
(pid=26444) File cars03Cropped_d2.csv was not processed for columns (d1,d13).
(pid=26444) Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.
(pid=26444) 
(pid=26444) cars03Cropped_d2.csv (len: 

(pid=26455) File efashion.csv was not processed for columns (year,d4).
(pid=26455) float division by zero
(pid=26455) 
(pid=26446) TopologyException: Input geom 0 is invalid: Self-intersection at or near point 0.26003888471902009 0.0054262413766240549 at 0.26003888471902009 0.0054262413766240549
(pid=26446) File grid6_5d.csv was not processed for columns (pca2,d3).
(pid=26446) No Shapely geometry can be created from null value
(pid=26446) 
(pid=26446) grid6_5d.csv (len: 7776): (pca2-d4)
(pid=26446) Classes: 32
(pid=26441) cars03Cropped_d1.csv (len: 7404): (d6-d12)
(pid=26453) TopologyException: Input geom 1 is invalid: Self-intersection at or near point 0.49959827543151342 -0.011656310103003923 at 0.49959827543151342 -0.011656310103003923
(pid=26453) File swanson.csv was not processed for columns (pca1,b_fire).
(pid=26453) No Shapely geometry can be created from null value
(pid=26453) 
(pid=26453) swanson.csv (len: 1875): (pca1-r_fitness)
(pid=26448) cars03Cropped_d3.csv (len: 7404): (

(pid=26448) cars03Cropped_d3.csv (len: 7404): (d9-d12)
(pid=26448) Classes: 12
(pid=26444) cars03Cropped_d2.csv (len: 7404): (d7-d14)
(pid=26444) Classes: 53
(pid=26448) cars03Cropped_d3.csv (len: 7404): (d9-d13)
(pid=26448) Classes: 12
(pid=26450) TopologyException: Input geom 0 is invalid: Self-intersection at or near point 0.3333333333333332 -0.0044779511162935269 at 0.3333333333333332 -0.0044779511162935269
(pid=26450) File grid10_4d.csv was not processed for columns (pca1,d4).
(pid=26450) No Shapely geometry can be created from null value
(pid=26450) 
(pid=26450) grid10_4d.csv (len: 10000): (pca2-d2)
(pid=26450) Classes: 16
(pid=26441) cars03Cropped_d1.csv (len: 7404): (d7-d8)
(pid=26448) cars03Cropped_d3.csv (len: 7404): (d9-d14)
(pid=26448) Classes: 12
(pid=26444) cars03Cropped_d2.csv (len: 7404): (d8-d9)
(pid=26444) Classes: 53
(pid=26450) TopologyException: Input geom 0 is invalid: Self-intersection at or near point 0.61938444539869753 0.0043417204185702083 at 0.61938444539869

## Data Aquisition

In [24]:
from pydataset import data

def rename_and_save(name, col, folder='/home/morarica/Developer/SepMe/data/orig_data/input_data/Orig_data/'):
    df = data(name)
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df2 = df.select_dtypes(exclude = numerics)
    cols = df2.columns
    
    le = LabelEncoder()
    df['class'] = le.fit_transform(df[col])
    for c in cols:
        df[c] = le.fit_transform(df[c])
    
    
    df.drop(col,axis=1,inplace=True)
    df.to_csv(folder + name+'.csv', index=False)


In [25]:
for i,item in data().iterrows():
    if len(data(item['dataset_id']).columns)>3:
        print(i)
        print(item)
        print('')
        print(data(item['dataset_id']).head(3))
        print('-------------')
        print('')

4
dataset_id                                 HairEyeColor
title         Hair and Eye Color of Statistics Students
Name: 4, dtype: object

    Hair    Eye   Sex  Freq
1  Black  Brown  Male    32
2  Brown  Brown  Male    53
3    Red  Brown  Male    10
-------------

8
dataset_id                        LifeCycleSavings
title         Intercountry Life-Cycle Savings Data
Name: 8, dtype: object

              sr  pop15  pop75      dpi  ddpi
Australia  11.43  29.35   2.87  2329.68  2.87
Austria    12.07  23.32   4.41  1507.99  3.93
Belgium    13.17  23.80   4.43  2108.47  3.82
-------------

10
dataset_id                OrchardSprays
title         Potency of Orchard Sprays
Name: 10, dtype: object

   decrease  rowpos  colpos treatment
1        57       1       1         D
2        95       2       1         E
3         8       3       1         B
-------------

13
dataset_id                                  Titanic
title         Survival of passengers on the Titanic
Name: 13, dtype: object

 

55
dataset_id                               quakes
title         Locations of Earthquakes off Fiji
Name: 55, dtype: object

     lat    long  depth  mag  stations
1 -20.42  181.62    562  4.8        41
2 -20.62  181.03    650  4.2        15
3 -26.00  184.10     42  5.4        43
-------------

58
dataset_id                                      rock
title         Measurements on Petroleum Rock Samples
Name: 58, dtype: object

   area     peri     shape  perm
1  4990  2791.90  0.090330   6.3
2  7002  3892.60  0.148622   6.3
3  7558  3930.66  0.183312   6.3
-------------

60
dataset_id                           stackloss
title         Brownlee's Stack Loss Plant Data
Name: 60, dtype: object

   Air.Flow  Water.Temp  Acid.Conc.  stack.loss
1        80          27          89          42
2        80          27          88          37
3        75          25          90          37
-------------

64
dataset_id                                                       swiss
title         Swiss F

139
dataset_id                   pneumon
title         data from Section 1.13
Name: 139, dtype: object

   chldage  hospital  mthage  urban  alcohol  smoke  region  poverty  bweight  \
1     12.0         0      22      1        0      0       1        1        1   
2     12.0         0      20      1        1      0       1        1        0   
3      3.0         0      24      1        3      0       1        1        0   

   race  education  nsibs  wmonth  sfmonth  agepn  
1     1         10      1       1        1      1  
2     1         12      1       2        2     12  
3     1         12      2       1        0      3  
-------------

140
dataset_id                     psych
title         data from Section 1.15
Name: 140, dtype: object

   sex  age  time  death
1    2   51     1      1
2    2   58     1      1
3    2   55     2      1
-------------

141
dataset_id                             rats
title         data from Exercise 7.13, p225
Name: 141, dtype: object

   litter  

Name: 182, dtype: object

      x1      x2     x3     x4     x5      y
1  0.573  0.1059  0.465  0.538  0.841  0.534
2  0.651  0.1356  0.527  0.545  0.887  0.535
3  0.606  0.1273  0.494  0.521  0.920  0.570
-------------

183
dataset_id                            AMSsurvey
title         American Math Society Survey Data
Name: 183, dtype: object

    type     sex citizen  count  count11
1  I(Pu)    Male      US    132      148
2  I(Pu)  Female      US     35       40
3  I(Pr)    Male      US     87       63
-------------

185
dataset_id                                  Angell
title         Moral Integration of American Cities
Name: 185, dtype: object

           moral  hetero  mobility region
Rochester   19.0    20.6      15.0      E
Syracuse    17.0    15.6      20.2      E
Worcester   16.4    22.1      13.6      E
-------------

186
dataset_id                                  Anscombe
title         U. S. State Public-School Expenditures
Name: 186, dtype: object

    education  income  

  work  hoursw  child6  child618  agew  educw  hearnw  wagew  hoursh  ageh  \
1   no    1610       1         0    32     12  3.3540   2.65    2708    34   
2   no    1656       0         2    30     12  1.3889   2.65    2310    30   
3   no    1980       1         3    35     12  4.5455   4.04    3072    40   

   educh   wageh  income  educwm  educwf  unemprate city  experience  
1     12  4.0288   16310      12       7        5.0   no          14  
2      9  8.4416   21800       7       7       11.0  yes           5  
3     12  3.5807   21040      12       7        5.0   no          15  
-------------

213
dataset_id                                   OBrienKaiser
title         O'Brien and Kaiser's Repeated-Measures Data
Name: 213, dtype: object

  treatment gender  pre.1  pre.2  pre.3  pre.4  pre.5  post.1  post.2  post.3  \
1   control      M      1      2      4      2      1       3       2       5   
2   control      M      4      4      5      3      4       2       2       3   

...

593
dataset_id                         PreSex
title         Pre-marital Sex and Divorce
Name: 593, dtype: object

  MaritalStatus ExtramaritalSex PremaritalSex Gender  Freq
1      Divorced             Yes           Yes  Women    17
2       Married             Yes           Yes  Women     4
3      Divorced              No           Yes  Women    54
-------------

594
dataset_id                  Punishment
title         Corporal Punishment Data
Name: 594, dtype: object

   Freq attitude memory   education    age
1     1       no    yes  elementary  15-24
2     3       no    yes  elementary  25-39
3    20       no    yes  elementary    40-
-------------

595
dataset_id                      RepVict
title         Repeat Victimization Data
Name: 595, dtype: object

         Rape  Assault  Robbery  Pickpocket  Personal Larcency  Burglary  \
Rape       26       50       11           6                 82        39   
Assault    65     2997      238          85               2553      1083   
Ro

634
dataset_id                           sanction
title         Multilateral Economic Sanctions
Name: 634, dtype: object

   mil  coop  target  import  export  cost  num          ncost
1    1     4       3       1       1     4   15     major loss
2    0     2       3       0       1     3    4    modest loss
3    0     1       3       1       0     2    1  little effect
-------------

635
dataset_id                                      sna.ex
title         Simulated Example of Social Network Data
Name: 635, dtype: object

Empty DataFrame
Columns: [Var1, Var2, Var3, Var4, Var5]
Index: []
-------------

636
dataset_id                                                       swiss
title         Swiss Fertility and Socioeconomic Indicators (1888) Data
Name: 636, dtype: object

              Fertility  Agriculture  Examination  Education  Catholic  \
Courtelary         80.2         17.0           15         12      9.96   
Delemont           83.1         45.1            6          9     84.84

691
dataset_id                                     Holzinger.9
title         Seven data sets showing a bifactor solution.
Name: 691, dtype: object

          vis_perc    cubes  lozenges  par_comp  sen_comp  wordmean  addition  \
vis_perc   1.00000  0.32580   0.44864   0.34163   0.30910   0.31713  0.104190   
cubes      0.32580  1.00000   0.41701   0.22800   0.15948   0.19465  0.066362   
lozenges   0.44864  0.41701   1.00000   0.32795   0.28685   0.34727  0.074638   

          count_dot  s_c_caps  
vis_perc    0.30760   0.48683  
cubes       0.16796   0.24786  
lozenges    0.23857   0.37258  
-------------

692
dataset_id                                           Reise
title         Seven data sets showing a bifactor solution.
Name: 692, dtype: object

            phone   routine   illness    listen   explain   respect      time  \
phone    1.000000  0.439269  0.424525  0.504945  0.403807  0.482973  0.475139   
routine  0.439269  1.000000  0.386764  0.431826  0.358271  0.408604  0.421

704
dataset_id                                                                               cubits
title         Galton's example of the relationship between height and 'cubit' or forearm length
Name: 704, dtype: object

    16.5  16.75  17.25  17.75  18.25  18.75  19.25  19.75
71     0      0      0      1      3      4     15      7
70     0      0      0      1      5     13     11      0
69     0      1      1      2     25     15      6      0
-------------

705
dataset_id                                                                                       cushny
title         A data set from Cushny and Peebles (1905) on the effect of three drugs on hours of sle...
Name: 705, dtype: object

   Control  drug1  drug2L  drug2R  delta1  delta2L  delta2R
1      0.6    1.3     2.5     2.1     0.7      1.9      1.5
2      3.0    1.4     3.8     4.4    -1.6      0.8      1.4
3      4.7    4.5     5.8     4.7    -0.2      1.1      0.0
-------------

706
dataset_id                        

   id    y  sex  age  trt  baseline  time
1   1  4.0    2   54    2         2     1
2   1  5.0    2   54    2         2     3
3   1  5.0    2   54    2         2     5
-------------

739
dataset_id          housing
title         Homeless Data
Name: 739, dtype: object

   id    y  time  sec
1   1  1.0     0    1
2   1  2.0     6    1
3   1  2.0    12    1
-------------

749
dataset_id                                                        InstEval
title         University Lecture/Instructor Evaluations by Students at ETH
Name: 749, dtype: object

   s     d  studage  lectage  service  dept  y
1  1  1002        2        2        0     2  5
2  1  1050        2        1        1     6  2
3  1  1582        2        2        0     2  5
-------------

750
dataset_id                              Pastes
title         Paste strength by batch and cask
Name: 750, dtype: object

   strength batch cask sample
1      62.8     A    a    A:a
2      62.6     A    a    A:a
3      60.1     A    b    A:b
-

In [30]:
names = ['OrchardSprays', 'Puromycin', 'Titanic', 'UCBAdmissions', 'attenu','esoph', 'sleep', 'aml','cane','kidney','CrohnD']
cols = ['treatment', 'state', 'Class' ,'Admit', 'station','agegp', 'ID', 'group', 'block','disease','treat']


for i in range(len(names)):
    try:
        rename_and_save(names[i], cols[i])
    except:
        print(names[i])
        continue

attenu
